In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import mplcursors
import seaborn as sns
from shapely.geometry import Point
import geopy
from geopy.geocoders import Nominatim
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster
from folium import plugins
#matplotlib.use('TkAgg')

In [134]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [135]:
geolocator = Nominatim(user_agent="Jupyter_Notebook")

In [136]:
def get_lat_long_from_zip(zip_code):
    try:
        location = geolocator.geocode(zip_code)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return get_lat_long_from_zip(zip_code)  # retry on timeout

In [137]:
df = pd.read_csv('/content/Sheet 2_data.csv')

In [138]:
# Process the data (ensuring the ZIP code is in the correct format)
df['Rndrng Prvdr Zip5'] = df['Rndrng Prvdr Zip5'].astype(str).str.zfill(5)

In [154]:
m = folium.Map(location=[39.8283, -98.5795], zoom_start=5)

In [155]:
# Function to choose color based on classification
def choose_color(classification):
    return 'blue' if classification == 'urban' else 'red'

In [156]:
l=[]
l= df['Rndrng Prvdr Zip5'].tolist()

In [157]:
df['Rndrng Prvdr Zip5']=l

In [158]:
for i in range(len(l)):
    l[i]=l[i] + ', USA'

In [39]:
lt=[]
lo=[]
for i in range(len(l)):
    # We need to get the latitude and longitude for each ZIP code
    # This is a placeholder as the actual method to get lat, long from ZIP code will depend on your data source
    lat, long = get_lat_long_from_zip(l[i])
    lt.append(lat)
    lo.append(long)

In [159]:
# Function to scale the payment amount for icon size
def scale_icon_size(amount, scale_factor=0.0000001, min_size=0.5, max_size=5):
    size = max(min(amount * scale_factor, max_size), min_size)
    return int(size), int(size)

In [160]:

# Adding markers with BeautifyIcon
for i in range(len(l)):
    lat = (lt[i])
    long = (lo[i])
    if lat == None:
        continue
    icon_size = scale_icon_size(df.iloc[i]['Tot Mdcr Pymt Amt'])
    icon = plugins.BeautifyIcon(
        icon_shape='circle-dot',
        border_color=choose_color(df.iloc[i]['Classification']),
        border_width=icon_size[0],
        text_color=choose_color(df.iloc[i]['Classification']),
        inner_icon_style=f'margin-top:0; opacity: 0.5;'  # Adjusting the opacity here
        )
    folium.Marker(
            location=[lat, long],
            icon=icon,
            popup=f"{df.iloc[i]['Rndrng Prvdr Zip5']}: ${df.iloc[i]['Tot Mdcr Pymt Amt']}"
        ).add_to(m)

# Display the map
m
